## example-ast-1
### AST
- AST(Abstract syntax tree，抽象语法树)

### REF
#### 相关知识点
- [`go/token` package](http://golang.org/pkg/go/token)
defines the lexical tokens of Go.
- The [`go/scanner` package](http://golang.org/pkg/go/scanner) tokenizes an input stream and records
file position information for use in diagnostics
or for file surgery in a refactoring tool.
- The [`go/ast` package](http://golang.org/pkg/go/ast)
defines the data types of the abstract syntax tree (AST).
- The [`go/parser` package](http://golang.org/pkg/go/parser)


### e.g. 把 go 源码解析为抽象语法树（AST)

- go 语言把语言代码的编译过程及步骤，分离出了几个对应的模块包

In [1]:
import "go/ast"
import "go/token"
import "go/parser"

- 一段 go 语言代码，也就是一个字符串
- 语言代码虽然是有结构的，但终归是一长串字符，如果计算机要处理源代码，从编译过程计算机要干的活看，字符串结构形式的数据处理确实不太方便

In [6]:
src := `package main 
import "fmt"
func main() {
    fmt.Println("hello")
}`

- 树是计算机常见的一种数据组织结构
- 语言编译器把源代码从字符串形式“抽象”转化为树结构的形式，表达语言语法内容是一致的，只是形式不一样

In [7]:
fset := token.NewFileSet()
// 这句代码包含了，解析源码字符串，提前“token"，生成 AST 语法树的过程
f, err := parser.ParseFile(fset, "", src, 0)
if err != nil {
    panic(err)
}
// 打印输出源码对应的 AST 语法树结构
ast.Print(fset, f)

     0  *ast.File {
     1  .  Package: 1:1
     2  .  Name: *ast.Ident {
     3  .  .  NamePos: 1:9
     4  .  .  Name: "main"
     5  .  }
     6  .  Decls: []ast.Decl (len = 2) {
     7  .  .  0: *ast.GenDecl {
     8  .  .  .  TokPos: 2:1
     9  .  .  .  Tok: import
    10  .  .  .  Lparen: -
    11  .  .  .  Specs: []ast.Spec (len = 1) {
    12  .  .  .  .  0: *ast.ImportSpec {
    13  .  .  .  .  .  Path: *ast.BasicLit {
    14  .  .  .  .  .  .  ValuePos: 2:8
    15  .  .  .  .  .  .  Kind: STRING
    16  .  .  .  .  .  .  Value: "\"fmt\""
    17  .  .  .  .  .  }
    18  .  .  .  .  .  EndPos: -
    19  .  .  .  .  }
    20  .  .  .  }
    21  .  .  .  Rparen: -
    22  .  .  }
    23  .  .  1: *ast.FuncDecl {
    24  .  .  .  Name: *ast.Ident {
    25  .  .  .  .  NamePos: 3:6
    26  .  .  .  .  Name: "main"
    27  .  .  .  .  Obj: *ast.Object {
    28  .  .  .  .  .  Kind: func
    29  .  .  .  .  .  Name: "main"
    30  .  .  .  .  .  Decl: *(obj @ 23)
    31  .  .  .  . 

### 按字面理解注释下 AST 结构
AST 树在 go 编译器内使用类似 map 的数据结构来存储
```json
     0  *ast.File {           // 作为语法树的根节点，对应这一个源码文件，在 go/ast 模块中有个 File 的数据结构来描述相关信息
     1  .  Package: 1:1       // package token 在源码中的位置[行号:字符偏移]，第1行第1个字符开始
     2  .  Name: *ast.Ident { // main token 使用 go/ast 中 Ident 的结构来描述
     3  .  .  NamePos: 1:9    // 第1行第9个字符开始
     4  .  .  Name: "main"
     5  .  }
     6  .  Decls: []ast.Decl (len = 2) { // 在编译原理中，这个叫”声明“，说明有 2 个一级的声明， 使用 go/ast 中 Decl 描述
     7  .  .  0: *ast.GenDecl {  // 通用声明， 比如语言内置的关键字声明等
     8  .  .  .  TokPos: 2:1     // 第2行第1个字符开始
     9  .  .  .  Tok: import
    10  .  .  .  Lparen: -
    11  .  .  .  Specs: []ast.Spec (len = 1) {
    12  .  .  .  .  0: *ast.ImportSpec { // 描述 import 关键字语法
    13  .  .  .  .  .  Path: *ast.BasicLit { // import 包的路径 
    14  .  .  .  .  .  .  ValuePos: 2:8
    15  .  .  .  .  .  .  Kind: STRING
    16  .  .  .  .  .  .  Value: "\"fmt\""
    17  .  .  .  .  .  }
    18  .  .  .  .  .  EndPos: -
    19  .  .  .  .  }
    20  .  .  .  }
    21  .  .  .  Rparen: -
    22  .  .  }
    23  .  .  1: *ast.FuncDecl { // 函数声明，func main()
    24  .  .  .  Name: *ast.Ident { // 函数名，程序员可以命名的一般用 go/ast Ident 结构描述
    25  .  .  .  .  NamePos: 3:6
    26  .  .  .  .  Name: "main"
    27  .  .  .  .  Obj: *ast.Object {
    28  .  .  .  .  .  Kind: func
    29  .  .  .  .  .  Name: "main"
    30  .  .  .  .  .  Decl: *(obj @ 23)
    31  .  .  .  .  }
    32  .  .  .  }
    33  .  .  .  Type: *ast.FuncType { // 函数类型
    34  .  .  .  .  Func: 3:1
    35  .  .  .  .  Params: *ast.FieldList { // 参数列表
    36  .  .  .  .  .  Opening: 3:10 // 左括号的位置
    37  .  .  .  .  .  Closing: 3:11 // 右括号的位置
    38  .  .  .  .  }
    39  .  .  .  }
    40  .  .  .  Body: *ast.BlockStmt { // 函数体
    41  .  .  .  .  Lbrace: 3:13 // 函数体的起始字符位置，左大括号 '{' 的位置
    42  .  .  .  .  List: []ast.Stmt (len = 1) { // 是个列表结构，描述着一行行的代码, 一般称”语句"
    43  .  .  .  .  .  0: *ast.ExprStmt { // 表达式
    44  .  .  .  .  .  .  X: *ast.CallExpr { // 调用形式的表达
    45  .  .  .  .  .  .  .  Fun: *ast.SelectorExpr { // fmt.Println
    46  .  .  .  .  .  .  .  .  X: *ast.Ident {
    47  .  .  .  .  .  .  .  .  .  NamePos: 4:5
    48  .  .  .  .  .  .  .  .  .  Name: "fmt"
    49  .  .  .  .  .  .  .  .  }
    50  .  .  .  .  .  .  .  .  Sel: *ast.Ident {
    51  .  .  .  .  .  .  .  .  .  NamePos: 4:9
    52  .  .  .  .  .  .  .  .  .  Name: "Println"
    53  .  .  .  .  .  .  .  .  }
    54  .  .  .  .  .  .  .  }
    55  .  .  .  .  .  .  .  Lparen: 4:16
    56  .  .  .  .  .  .  .  Args: []ast.Expr (len = 1) { // 参数列表
    57  .  .  .  .  .  .  .  .  0: *ast.BasicLit {
    58  .  .  .  .  .  .  .  .  .  ValuePos: 4:17
    59  .  .  .  .  .  .  .  .  .  Kind: STRING
    60  .  .  .  .  .  .  .  .  .  Value: "\"hello\""
    61  .  .  .  .  .  .  .  .  }
    62  .  .  .  .  .  .  .  }
    63  .  .  .  .  .  .  .  Ellipsis: -
    64  .  .  .  .  .  .  .  Rparen: 4:24
    65  .  .  .  .  .  .  }
    66  .  .  .  .  .  }
    67  .  .  .  .  }
    68  .  .  .  .  Rbrace: 5:1 // 函数体结束的字符位置，右大括号 '}' 的位置
    69  .  .  .  }
    70  .  .  }
    71  .  }
    72  .  Scope: *ast.Scope { // 作用域，这个代码的 AST 树是在 main 包的范围内这么理解
    73  .  .  Objects: map[string]*ast.Object (len = 1) {
    74  .  .  .  "main": *(obj @ 27)
    75  .  .  }
    76  .  }
    77  .  Imports: []*ast.ImportSpec (len = 1) { // 记录此源码导入引用的外部包
    78  .  .  0: *(obj @ 12)
    79  .  }
    80  .  Unresolved: []*ast.Ident (len = 1) { // 源码中没有解析的标识符
    81  .  .  0: *(obj @ 46)
    82  .  }
    83  }
```

- 从上面的 ast.File 结构看，第一级的内容主要是源码模块化方面的信息，描述 package 与 package 之间的信息
- 从上面输出的信息看，一个源码文件中源代码的语言文法结构实际从 ast.Decl 声明开始
                        ast.Decl
            +--------------+--------------------+
        ast.GenDecl    ast.FuncDecl